In [ ]:
# plt.show()で可視化されない人はこのセルを実行してください。
%matplotlib inline

In [ ]:
# 初めの一回だけこのセルを実行してください、データセットをダウンロードして展開します
# 一回実行すれば、データセットはダウンロードされたままなので、再起動後等再び実行する必要はありません
import urllib.request
import zipfile

# URLを指定
url = "https://storage.googleapis.com/tutor-contents-dataset/5030_unsupervised_learning_data.zip"
save_name = url.split('/')[-1]

# ダウンロードする
mem = urllib.request.urlopen(url).read()

# ファイルへ保存
with open(save_name, mode='wb') as f:
    f.write(mem)

# zipファイルをカレントディレクトリに展開する
zfile = zipfile.ZipFile(save_name)
zfile.extractall('.')

#  主成分分析

- **[3.1 主成分分析](#3.1-主成分分析)**
    - **[3.1.1 主成分分析](#3.1.1-主成分分析)**
    - **[3.1.2 特徴変換までの流れ](#3.1.2-特徴変換までの流れ)**
    - **[3.1.3 データの用意](#3.1.3-データの用意)**
    - **[3.1.4 標準化](#3.1.4-標準化)**
    - **[3.1.5 相関行列の計算](#3.1.5-相関行列の計算)**
    - **[3.1.6 固有値分解](#3.1.6-固有値分解)**
    - **[3.1.7 特徴変換](#3.1.7-特徴変換)**
    - **[3.1.8 scikit-learnを使った主成分分析](#3.1.8-scikit-learnを使った主成分分析)**
    - **[3.1.9 前処理としての主成分分析](#3.1.9-前処理としての主成分分析)**
<br><br>
- **[3.2 カーネル主成分分析](#3.2-カーネル主成分分析)**
    - **[3.2.1 カーネル主成分分析](#3.2.1-カーネル主成分分析)**
    - **[3.2.2 カーネルトリックⅠ](#3.2.2-カーネルトリックⅠ)**
    - **[3.2.3 カーネルトリックⅡ](#3.2.3-カーネルトリックⅡ)**
    - **[3.2.4 特徴変換](#3.2.4-特徴変換)**
    - **[3.2.5 scikit-learnを使ったカーネル主成分分析](#3.2.5-scikit-learnを使ったカーネル主成分分析)**
<br><br>
- **[3.3 まとめ問題(提出不要)](#3.3-まとめ問題(提出不要))**

***

## 3.1 主成分分析

### 3.1.1 主成分分析

　<b style='color: #AA0000'>主成分分析</b>(Principal Component Analysis : <b style='color: #AA0000'>PCA</b>) は、**データの要約** （少ないデータ数で元のデータを表現すること）の強力な手法のひとつです。  

　例として、主成分分析を用いて10人の学生の数学と国語の得点データ（2次元）を圧縮し1次元に変換するイメージを示します。

<img src="https://aidemyexcontentspic.blob.core.windows.net/contents-pic/5030_unsupervised_learning/unsupervised_chap3_3.png">

<b><center>図3.1.1-1 主成分分析のイメージ図</center></b>

　右の図のように数学の得点だけ（1次元）で個々人の点数を説明するより、左の図のように新しい軸を1つ用意し新たに1次元データを作った方が小さい誤差で説明できます。  
　左の図は主成分分析を用いたデータ圧縮の図です。**主成分分析**を用いると、**全部のデータを最も効率よく説明できるような軸（<b style='color: #AA0000'>第一主成分</b>の軸）**と**それだけでは説明しきれないデータを最も効率よく説明する軸（<b style='color: #AA0000'>第二主成分</b>の軸）**が作られます。  
第一主成分は元のデータをよく表現できるので、第二主成分の情報を捨てる（使わない）ことで**効率よくデータの圧縮ができます**。

　主成分分析の実用例として、製品やサービスのスコアリングや比較（1次元に圧縮）、データの可視化（2,3次元に圧縮）、回帰分析の前処理、などがあげられます。  
**主成分分析は実用性が高く、機械学習の分野において重要なテーマの一つとなっています**。

#### 問題

- 主成分分析について述べた文として**正しいものを選んでください**。

- 主成分分析は、教師あり学習の一つの手法である。
- 主成分分析を使う際、主成分となる軸を予め求めておく必要がある。
- 主成分分析を用いると、効率よくデータの圧縮ができる。
- 主成分分析を用いると必ずデータ数が減るため、実用的ではない。

#### ヒント

- 主成分分析は教師あり学習の前処理として使われることはありますが、主成分分析自体は教師あり学習に分類されません。

#### 解答

- 主成分分析を用いると、効率よくデータの圧縮ができる。

***

### 3.1.2 特徴変換までの流れ

　主成分分析を使って、以下の手順で　<b style='color: #AA0000'>データの圧縮（特徴変換）</b>を行います。  
　少し難しい内容になっていますが問題ありません。次のセッションから詳しく見ていきます。

- データを**標準化**します。
- 特徴同士の**相関行列**を計算します。
- 相関行列の**固有ベクトルと固有値**を求めます。
- 得られた固有値を大きい方からk個選び、対応する**固有ベクトルを選択**します。このkの値は、圧縮したい次元数を指定してください。
- k個の固有ベクトルから**特徴変換行列** ${W}$ を作成します。
- d次元のデータ ${X}$ と行列 ${W}$ で行列の積をとり、**k次元に変換されたデータ** ${X'}$ を得ます。

　以下の図は、このチャプターで扱うワインのデータセットを <b style='color: #AA0000'>特徴変換</b>し、データを13次元から2次元に要約したイメージです。  

<img src="https://aidemyexcontentspic.blob.core.windows.net/contents-pic/5030_unsupervised_learning/unsupervised_chap3_2.png">

#### 問題

- 主成分分析を用いた特徴変換について述べた文として、 *最も適切なもの* を選んでください。ただし、データ数（行数）をn,m、成分数（列数）をk,lとします。

- データ数n、成分数lのデータを特徴変換すると、データ数m（≠n）、成分数k（≠l）のデータができる。
- データ数n、成分数lのデータを特徴変換すると、データ数n、成分数k（≠l）のデータができる。
- データ数n、成分数lのデータを特徴変換すると、データ数m（≠n）、成分数1のデータができる。
- データ数n、成分数lのデータを特徴変換すると、データ数n、成分数1のデータができる。

#### ヒント

- 主成分分析を適切に適応させることにより、特徴量を欲しい次元に要約したデータを得ることができます。

#### 解答

- データ数n、成分数lのデータを特徴変換すると、データ数n、成分数k（≠l）のデータができる。

***

### 3.1.3 データの用意

　実際に**主成分分析**を行っていきましょう。用いるデータは、　**「UCI Machine Learning Repository」**　で公開されているワインのデータです。　**178行**分のワインサンプルに対して、**ぶどうの種類データ**（1〜3のラベル）と**ワインの化学的性質を表す特徴量データ**（13種類）で構成されています。

　以下のようにしてデータを取得します。
```python
import pandas as pd
df_wine = pd.read_csv("./5030_unsupervised_learning_data/wine.csv", header = None)
# 特徴量データをXに、ラベルデータをyに格納。
# df_wineの1列目はラベルデータ、2列目以降は特徴量データです。
X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values
```

#### 問題

- 空欄を埋めてワインのデータを用意し、 **データ数と各データの特徴量の数を取得、表示** してください。

In [ ]:
import pandas as pd

df_wine = pd.read_csv("./5030_unsupervised_learning_data/wine.csv", header=None)

X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values

# ここに回答を記述してください


#### ヒント

`X`, `y` はnumpy配列です。  
numpy配列 `A` のサイズは以下のようにして取得、表示できます。
```python
print(A.shape)
```

#### 解答例

In [ ]:
import pandas as pd

df_wine = pd.read_csv("./5030_unsupervised_learning_data/wine.csv", header=None)

X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values

# ここに回答を記述してください
print(X.shape)

***

### 3.1.4 標準化

予めワインのデータを、各特徴量について平均が0、分散が1となるように変換します。これを <b style='color: #AA0000'>標準化</b>と言います。  
　<b style='color: #AA0000'>標準化</b>を行うことによって、アルコール度数やワインの色相など、単位も基準となる値もバラバラな **様々な種類のデータを同じように扱うことが可能になります**。

```python
import numpy as np
# 標準化
X = (X - X.mean(axis=0)) / X.std(axis=0)
```

#### 問題

- 空欄を埋めて **標準化** を行ってください。また結果を見て **次の2点を確認してください。**
 - 標準化によって平均がほぼ0に分散が1になっており、かつ分布の形は変わっていないこと。
 - 特徴量同士の関連性が明らかに高そうなものがあること。（今後、関連性の高いデータ同士をまとめることによってデータを圧縮していきます。）

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df_wine = pd.read_csv("./5030_unsupervised_learning_data/wine.csv", header=None)
X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values

# 標準化前のデータを可視化
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 3))
ax1.set_title('before')
ax2.set_title('before')
ax1.scatter(X[:, 0], X[:, 1])
ax2.scatter(X[:, 5], X[:, 6])
plt.show()

print("before")
print("mean: ", X.mean(axis=0), "\nstd: ", X.std(axis=0))

# Xに、Xを標準化したデータを代入してください
X =

# 標準化後のデータを可視化
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 3))
ax1.set_title('after')
ax2.set_title('after')
ax1.scatter(X[:, 0], X[:, 1])
ax2.scatter(X[:, 5], X[:, 6])
plt.show()

print("after")
print("mean: ", X.mean(axis=0), "\nstd: ", X.std(axis=0))

#### ヒント

`X` はnumpy配列です。  
`X`を標準化するには以下のようにします。
```python
X = (X - X.mean(axis=0)) / X.std(axis=0)
```

#### 解答例

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df_wine = pd.read_csv("./5030_unsupervised_learning_data/wine.csv", header=None)
X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values

# 標準化前のデータを可視化
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 3))
ax1.set_title('before')
ax2.set_title('before')
ax1.scatter(X[:, 0], X[:, 1])
ax2.scatter(X[:, 5], X[:, 6])
plt.show()

print("before")
print("mean: ", X.mean(axis=0), "\nstd: ", X.std(axis=0))

# Xに、Xを標準化したデータを代入してください
X = (X - X.mean(axis=0)) / X.std(axis=0)

# 標準化後のデータを可視化
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 3))
ax1.set_title('after')
ax2.set_title('after')
ax1.scatter(X[:, 0], X[:, 1])
ax2.scatter(X[:, 5], X[:, 6])
plt.show()

print("after")
print("mean: ", X.mean(axis=0), "\nstd: ", X.std(axis=0))

***

### 3.1.5 相関行列の計算

　<b>特徴量ごとの類似度</b>を調べるため、データの <b style='color: #AA0000'>相関行列</b>を計算します。  

　<b>相関係数</b>は、2つのデータ間の<b>直線的な関係性の強さ</b>を表す指標で -1〜1の値をとります。  
　<b>相関係数が1に近い</b>（正の相関が強い）ときは、左の図のように<b>二つのデータは片方が増加するともう片方も増加する</b> ような一次関数的な分布をとります。<br>
　<b>負の相関が強い</b> ときは <b>片方が増加するともう片方は減少する</b> ような一次関数的な分布をとります。<br>
　<b>相関係数が0に近い</b> ときは、右の図のように <b>直線的な関係性があまり見られません。</b>
 
<img src="https://aidemyexcontentspic.blob.core.windows.net/contents-pic/5030_unsupervised_learning/unsupervised_chap3_1.png" width=500>
 
　ここでは、ワインの13種類の特徴データについて、それぞれの相関係数を保持する13x13の相関行列を求めます。求める <b>相関行列</b> は以下のような形をしています。

$$
R = 
  \left(
    \begin{array}{ccc}
      r_{1,1} & r_{1,2} & \cdots & r_{1,13} \\
      r_{2,1} & r_{2,2} & \cdots & r_{2,13} \\
      \vdots & \vdots & \ddots & \vdots \\
      r_{13,1} & r_{2,13} & \cdots & r_{13,13} \\
    \end{array}
  \right)
$$


　以下のようにして <b>相関行列</b> を取得します。 corrcoef()関数は、列同士(横方向)のそれぞれの相関ではなく、行同士(縦方向)のそれぞれの相関を相関行列にします。そのため、このままではデータ同士の相関行列が求まってしまいます。<b>データ同士ではなく特徴同士の相関行列を求めるため</b>、X.Tで${X}$ を転置しています。
```python
import numpy as np
R = np.corrcoef(X.T)
```
　発展的な話になりますが、相関行列自体は標準化前の ${X}$ を用いても同様に計算できますが、この後のために予め標準化しました。  
　また、相関行列ではなく共分散行列を使った主成分分析もあります。
 
正方行列の単位行列を作成するコードは以下になります。

```python
import numpy as np

# np.identity(行列サイズ)
identity　= np.identity(3)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])
```

#### 問題

- 空欄を埋めて相関行列を求めてください。
- また、最も相関の高い特徴の組み合わせを調べ、コード中のコメント文に従ってその相関係数を出力してください。

In [ ]:
import pandas as pd
import numpy as np

df_wine = pd.read_csv("./5030_unsupervised_learning_data/wine.csv", header=None)
X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values

# 相関行列（13x13）を作成
R = np.corrcoef(X.T)

# 対角成分を0にしてください
_R =

# 最大相関係数をとるインデックスを1つだけ取得してください
index =

print(R[index[0], index[1]])
print(index)

#### ヒント

`R` はndarrayです。  
ndarray配列 `A` の最大要素値のインデックスを全て取得、表示するには以下のようにします。
```python
print(np.where(A == A.max()))
```
なお、対角成分は1（つまり、 `R[i,i] == 1` ）ですが、これらは相関係数ではないので注意してください。  
例えば、以下の式で相関行列 `R` の対角成分を0にした `_R` を生成できます。
```python
_R = R - np.identity(13)
```

#### 解答例

In [ ]:
import pandas as pd
import numpy as np

df_wine = pd.read_csv("./5030_unsupervised_learning_data/wine.csv", header=None)
X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values

# 相関行列（13x13）を作成
R = np.corrcoef(X.T)

# 対角成分を0にしてください
_R = R - np.identity(13)

# 最大相関係数をとるインデックスを1つだけ取得してください
index = np.where(_R == _R.max())[0]

print(R[index[0], index[1]])
print(index)

***

### 3.1.6 固有値分解

　　次に、得られた相関行列に <b style='color: #AA0000'>固有値分解</b>という数学的手法を適用し、固有ベクトルと固有値を取得します。  
　固有値分解を行うと、もとの13x13次元の行列 ${R}$ は13個の特別な13次元ベクトル **（固有ベクトル）** $v_1$ 〜 $v_{13}$ と13個の特別な数 **（固有値）** $\lambda_1$ 〜 $\lambda_{13}$ に分解されます。直感的には、元の行列は **固有ベクトルの方向に情報が集中しており、対応する固有値は情報の集中の度合いを示している** と言えます。


　numpyを使って以下のように固有値分解を計算できます。 ${R}$ の固有値13個と固有ベクトル13個がそれぞれ、 `eigvals` と `eigvecs` に格納されます。
```python
import numpy as np
# 相関行列から固有対を取得。 numpy.linalg.eighはそれらを固有値の昇順で返す
eigvals, eigvecs = np.linalg.eigh(R)
```

　以下は発展的内容になるので読み飛ばしても構いません。  

　相関行列 ${R}$ 、固有ベクトルを並べた行列 ${V}$ と固有値を並べた対角行列 ${D}$ は以下の式を満たします。

$${R}{V} = {V}{D}$$

　要素を表すと以下のようになります。

$$
  \left(
    \begin{array}{ccc}
      r_{1,1} & r_{1,2} & \cdots & r_{1,13} \\
      r_{2,1} & r_{2,2} & \cdots & r_{2,13} \\
      \vdots & \vdots & \ddots & \vdots \\
      r_{13,1} & r_{2,13} & \cdots & r_{13,13} \\
    \end{array}
  \right)
  \left(
    \begin{array}{ccc}
      & & &  \\
      & & &  \\
      { v_{1}} & { v_{2}} & \cdots & { v_{13}} \\
      & & &  \\
      & & &  \\
    \end{array}
  \right) = 
  \left(
    \begin{array}{ccc}
      & & &  \\
      & & &  \\
      { v_{1}} & { v_{2}} & \cdots & { v_{13}} \\
      & & &  \\
      & & &  \\
    \end{array}
  \right)
  \left(
    \begin{array}{ccc}
      \lambda_{1} & 0 & \cdots & 0 \\
      0 & \lambda_{2} & \cdots & 0 \\
      \vdots & \vdots & \ddots & \vdots \\
      0 & \cdots & 0 & \lambda_{13} \\
    \end{array}
  \right)
$$

　相関行列の固有ベクトルが **主成分ベクトル** を表し、固有ベクトルの成分は **各特徴量の主成分への影響の様子** を表しています。また、 **大きな固有値に対応する固有ベクトルほど元の行列の構成に深く関わっています** 。  
　つまり、小さい固有値に対応する固有ベクトル（主成分ベクトル）を無視することで、特徴量を削減しつつ情報の損失を少なく抑えることができます。

#### 問題

- 空欄を埋めて固有値分解を行ってください。また出力される図を見て、固有値に大きな偏りがあることを確認してください。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df_wine = pd.read_csv("./5030_unsupervised_learning_data/wine.csv", header=None)
X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values

# 相関行列（13x13）を作成
R = np.corrcoef(X.T)

# 固有値分解
eigvals, eigvecs =

# 可視化
plt.bar(range(13), eigvals)
plt.title("distribution of eigvals")
plt.xlabel("index")
plt.ylabel("eigvals")
plt.show()

# 消さないでください。実行結果の確認に使います。
print(eigvals)

#### ヒント

- `numpy.linalg.eigh` は、小さい固有値から順に格納されたものをひとつ目の返り値として出力します。

#### 解答例

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df_wine = pd.read_csv("./5030_unsupervised_learning_data/wine.csv", header=None)
X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values

# 相関行列（13x13）を作成
R = np.corrcoef(X.T)

# 固有値分解
eigvals, eigvecs = np.linalg.eigh(R)

# 可視化
plt.bar(range(13), eigvals)
plt.title("distribution of eigvals")
plt.xlabel("index")
plt.ylabel("eigvals")
plt.show()

# 消さないでください。実行結果の確認に使います。
print(eigvals)

***

### 3.1.7 特徴変換

　前のセッションでは、相関行列を固有値と固有ベクトルに分解しました。次に、その中から最大の固有値と2番目に大きい固有値に対応する2つの固有ベクトルを用いて、<b>13次元の特徴量を2次元に変換する</b>13x2の行列 ${W}$ を作り、13次元特徴量を持つWineデータ ${X}$ を、<b>第一主成分</b>と<b>第二主成分</b>の2次元の特徴量のみを持つ<b>新たなWineデータ</b> ${X'}$ に変換します。<br>
 
　以下のようにして変換行列 ${W}$ を作ります。
 
```python
# 列方向で、固有値が最大のものと2番目に最大のものに対応する固有ベクトルを連結します
W = np.c_[eigvecs[:,-1], eigvecs[:,-2]]
```

　上記より、13行2列の行列が作成できました。さらに、この行列 ${W}$ と元のデータ${X}$ との積を取ることにより、 ${X}$ を圧縮した行列 ${X'}$ を生成できます。

$${X'} = {X}{W}$$
                    
　行列の積の計算は、以下のコードで実行されます。
```python
import numpy as np
X_pca = X.dot(W)
```

　以下は発展的な内容になるため読み飛ばして構いません。  

　固有ベクトルと固有値を用いて、以下の式も成り立ちます。  

$${ R}{ V'} = { V'}{ D'}$$

$$
  \left(
    \begin{array}{ccc}
      r_{1,1} & r_{1,2} & \cdots & r_{1,13} \\
      r_{2,1} & r_{2,2} & \cdots & r_{2,13} \\
      \vdots & \vdots& \ddots& \vdots \\
      r_{13,1} & r_{13,2} & \cdots & r_{13,13} \\
    \end{array}
  \right)
  \left(
    \begin{array}{ccc}
      & \\
      & \\
      { v_{1}} & { v_{2}} \\
      & \\
      & \\
    \end{array}
  \right) = 
  \left(
    \begin{array}{ccc}
      & \\
      & \\
      { v_{1}} & { v_{2}} \\
      & \\
      & \\
    \end{array}
  \right)
  \left(
    \begin{array}{ccc}
      \lambda_{1} & 0 \\
      0 & \lambda_{2} \\
    \end{array}
  \right)
$$  
  
　大きい方から2つの固有値を $\lambda_1, \lambda_2$ 、それに対応する固有ベクトルを ${v_1}, {v_2}$ とします。この ${v_1}, {v_2}$ が変換行列となります。  
　${R}$ に固有ベクトル ${v_1}$ を掛けると、 ${v_1}$ <b>方向によく伸びた（分散の大きい）新たなデータ</b>が得られ、${R}$ に固有ベクトル ${v_2}$ を掛けると、 ${v_1}$ <b>ベクトルとは直行する（</b> ${v_1}$ <b>では説明できない）、</b> ${v_2}$ <b>方向によく伸びた（分散の大きい）新たなデータ</b>が得られます。  
　同様に、 ${X}$ に 固有ベクトル ${v_1}, {v_2}$ を掛けると、<b>2つの直行する方向によく伸びた特徴データ</b>を得ることができます。

#### 問題

- 空欄を埋めて、2次元に縮約されたデータを確認しましょう。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df_wine = pd.read_csv("./5030_unsupervised_learning_data/wine.csv", header=None)
X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values

# 標準化
X = (X - X.mean(axis=0)) / X.std(axis=0)

# 相関行列の取得
R = np.corrcoef(X.T)

# 固有値分解
eigvals, eigvecs = np.linalg.eigh(R)

# 変換行列の取得
W =

# 特徴変換
X_pca =

# 可視化
color = ["r", "b", "g"]
marker = ["s", "x", "o"]
for label, color, marker in zip(np.unique(y), color, marker):
    plt.scatter(X_pca[y == label, 0], X_pca[y == label, 1],
                c=color, marker=marker, label=label)
plt.xlabel("PC 1")
plt.ylabel("PC 2")
plt.legend(loc="lower left")
plt.show()

print(X_pca)  # 消さないでください。実行結果の確認に使います。

#### ヒント

- 13次元のデータを2次元に圧縮したので、13次元のデータとは違い簡単に可視化できます。

#### 解答例

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df_wine = pd.read_csv("./5030_unsupervised_learning_data/wine.csv", header=None)
X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values

# 標準化
X = (X - X.mean(axis=0)) / X.std(axis=0)

# 相関行列の取得
R = np.corrcoef(X.T)

# 固有値分解
eigvals, eigvecs = np.linalg.eigh(R)

# 変換行列の取得
W = np.c_[eigvecs[:, -1], eigvecs[:, -2]]

# 特徴変換
X_pca = X.dot(W)

# 可視化
color = ["r", "b", "g"]
marker = ["s", "x", "o"]
for label, color, marker in zip(np.unique(y), color, marker):
    plt.scatter(X_pca[y == label, 0], X_pca[y == label, 1],
                c=color, marker=marker, label=label)
plt.xlabel("PC 1")
plt.ylabel("PC 2")
plt.legend(loc="lower left")
plt.show()

print(X_pca)  # 消さないでください。実行結果の確認に使います。

***

### 3.1.8 scikit-learnを使った主成分分析

　ここまで、一つずつステップを踏んで主成分分析を用いた特徴変換を実装しました。実は同様のことを、 `sklearn.decomposition` の `PCA` クラスを用いて簡単にできます。  
　以下のようにして**PCAクラス**を使います。
```python
from sklearn.decomposition import PCA
# 主成分数を指定して、PCAのインスタンスを生成。引数で変換後の次元数を指定します。
pca = PCA(n_components=2)
# データから変換モデルを学習し、変換する。
X_pca = pca.fit_transform(X)
```
　`fit_transform()` メソッドでは、内部で自動的に変換行列を生成しています。

#### 問題

- 空欄を埋めてPCAクラスを用いて特徴変換を行い、2次元に縮約されたデータを確認しましょう。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# PCAをインポートしてください
# ---------------------------

# ---------------------------

df_wine = pd.read_csv("./5030_unsupervised_learning_data/wine.csv", header=None)
X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values
X = (X - X.mean(axis=0)) / X.std(axis=0)

# 主成分分析のインスタンスを生成。主成分数は2としてください。
pca =

# データから変換モデルを学習し、変換する。
X_pca =

# 可視化
color = ["r", "b", "g"]
marker = ["s", "x", "o"]
for label, color, marker in zip(np.unique(y), color, marker):
    plt.scatter(X_pca[y == label, 0], X_pca[y == label, 1],
                c=color, marker=marker, label=label)
plt.xlabel("PC 1")
plt.ylabel("PC 2")
plt.legend(loc="lower left")
plt.show()

print(X_pca)  # 消さないでください。実行結果の確認に使います。

#### ヒント

- 細かい実装の違いによって、前セッションと比べると上下や左右に反転されたようにプロットされることがありますが、本質的に同じような特徴変換が行われていることがわかります。

#### 解答例

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# PCAをインポートしてください
# ---------------------------
from sklearn.decomposition import PCA
# ---------------------------

df_wine = pd.read_csv("./5030_unsupervised_learning_data/wine.csv", header=None)
X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values
X = (X - X.mean(axis=0)) / X.std(axis=0)

# 主成分分析のインスタンスを生成
pca = PCA(n_components=2)

# データから変換モデルを学習し、変換する。
X_pca = pca.fit_transform(X)

# 可視化
color = ["r", "b", "g"]
marker = ["s", "x", "o"]
for label, color, marker in zip(np.unique(y), color, marker):
    plt.scatter(X_pca[y == label, 0], X_pca[y == label, 1],
                c=color, marker=marker, label=label)
plt.xlabel("PC 1")
plt.ylabel("PC 2")
plt.legend(loc="lower left")
plt.show()

print(X_pca)  # 消さないでください。実行結果の確認に使います。

***

### 3.1.9 前処理としての主成分分析

　ここまで学んだことを応用して、**回帰分析の前処理に主成分分析を適用します**。予めデータを圧縮することで、外れ値などの外乱に強く**より汎用性の高い回帰分析モデルを生成できます**。  

　まず、データをトレーニングデータとテストデータに分割します。

```python
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
```

　特徴変換を行う際、トレーニングデータとテストデータで違う変換行列を求めて特徴変換を行ってしまうと、変換行列が異なってしまうために特徴変換後のデータを比較することができません。標準化についても同じことが言えます。  
　これでは不便なことがあるので、**標準化と主成分分析を行う際はトレーニングデータとテストデータで共通の基準を使います**。

　**標準化**する際は、以下のように `StandardScalar` クラスを用いると便利です。

```python
from sklearn.preprocessing import StandardScaler
# 標準化のためのインスタンスを生成
sc = StandardScaler()
# トレーニングデータから変換モデルを学習し、テストデータに同じモデルを適用
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)
```

　**主成分分析**する際は、 `PCA` クラスを以下のように使います。

```python
from sklearn.decomposition import PCA
# 主成分分析のインスタンスを生成
pca = PCA(n_components=2)
# トレーニングデータから変換モデルを学習し、テストデータに同じモデルを適用
X_train_pca = pca.fit_transform(X_train_std)
X_test_pca = pca.transform(X_test_std)
```

復習となりますが、回帰分析は以下のように行います。

```python
from sklearn.linear_model import LogisticRegression
# ロジスティック回帰のインスタンスを生成
lr = LogisticRegression()
# 分類モデルを学習
lr.fit(X, y)
# スコアの表示
print(lr.score(X, y))
```

#### 問題

- 空欄を埋めてWineデータを2次元のデータに圧縮したあと、ロジスティック回帰を使用して分類しましょう。
- 次元を13から2に減らしたにも関わらず高い分類精度を出せていることを確認してください。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

df_wine = pd.read_csv("./5030_unsupervised_learning_data/wine.csv", header=None)

X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=0)

# 標準化のためのインスタンスを生成
sc =
# トレーニングデータから変換モデルを学習し、テストデータに適用
X_train_std =
X_test_std =

# 主成分分析のインスタンスを生成
pca =
# トレーニングデータから変換モデルを学習し、テストデータに適用
X_train_pca =
X_test_pca =

# ロジスティック回帰のインスタンスを生成
lr = LogisticRegression()
# 次元削減後のトレーニングデータで分類モデルを学習
lr.fit(X_train_pca, y_train)

# スコアの表示
print(lr.score(X_train_pca, y_train))
print(lr.score(X_test_pca, y_test))

#### ヒント

- 次元削除したデータをモデルの学習に使うと、若干ではありますが一般的にモデルの学習率が低下してしまいます。

#### 解答例

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

df_wine = pd.read_csv("./5030_unsupervised_learning_data/wine.csv", header=None)

X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=0)

# 標準化のためのインスタンスを生成
sc = StandardScaler()
# トレーニングデータから変換モデルを学習し、テストデータに適用
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

# 主成分分析のインスタンスを生成
pca = PCA(n_components=2)
# トレーニングデータから変換モデルを学習し、テストデータに適用
X_train_pca = pca.fit_transform(X_train_std)
X_test_pca = pca.transform(X_test_std)

# ロジスティック回帰のインスタンスを生成
lr = LogisticRegression()
# 次元削減後のトレーニングデータで分類モデルを学習
lr.fit(X_train_pca, y_train)

# スコアの表示
print(lr.score(X_train_pca, y_train))
print(lr.score(X_test_pca, y_test))

***

## 3.2 カーネル主成分分析

### 3.2.1 カーネル主成分分析

　回帰分析など、機械学習の多くのアルゴリズムは線形分離できるデータが与えられることを前提としています。ただ、現実問題として線形分離することが困難なデータ、つまり**非線形分離する必要があるデータがほとんどです**。このセッションでは、非線形分離する必要があるデータに対処できるカーネル化されたPCA、 **「カーネルPCA(kernel PCA)」** について本項では取り上げます。

　**カーネルPCA** ではまずはじめに、与えられた$N$x$M$（データ数x特徴の種類）のデータ ${X}$ を、**全く新しい**$N$x$M´$（データ数x特徴の種類）**のデータ** ${K}$ **に作り変えます <b style='color: #AA0000'>（カーネルトリック）</b>。** **カーネルトリック**を用いると一般的に特徴の種類は多くなり（特徴量が展開され）、**線形分離させやすくなります**。非線形性の高いデータに対して主成分分析を用いてもうまく行かないことが知られていますが、データをカーネル行列 ${K}$ に展開することで主成分分析ができるようになります。
 
　下の図は、円形に分布する2次元のデータに、カーネルトリックを使い特徴量を増やした後主成分分析を行って特徴量を2つに戻してプロットした図です。カーネルPCAを用いると、以下のような2次元空間で**線形分離不可能なデータ**を**線形分離可能なデータに変換する**ことができます。
 
<img src="https://aidemyexcontentspic.blob.core.windows.net/contents-pic/5030_unsupervised_learning/unsupervised_chap3_4.png">

#### 問題

- カーネル主成分分析について述べた文として最も適切なものを選んでください。

- カーネル主成分分析は、特徴量の次元を1次元に圧縮することを目的としている。
- カーネル主成分分析は、特徴量の次元を無限次元に展開することを目的としている。
- カーネル主成分分析は、線形分離可能な特徴量を線形分離不可能なものに変換することを目的としている。
- カーネル主成分分析は、線形分離不可能な特徴量を線形分離可能なものに変換することを目的としている。

#### ヒント

- カーネル主成分分析を適切に適応させることにより、欲しい次元分の線形分離可能な特徴量に要約したデータを得ることができます。

#### 解答

- カーネル主成分分析は、線形分離不可能な特徴量を線形分離可能なものに変換することを目的としている。

***

### 3.2.2 カーネルトリックⅠ

　まず <b style='color: #AA0000'>カーネル(類似度)行列</b>${K}$ を計算します。以下のような行列を**カーネル行列**といい、**全てのサンプルデータのペアごとに類似度を計算しています**。$N$x$M$（データ数x特徴の種類）のデータ ${X}$ のカーネル行列は、$N$x$N$ （データ数xデータ数）になります。**カーネル行列** ${K}$ **を、** ${X}$ **に代わる新たな**$N$x$M$**（データ数x特徴の種類）のデータ**と見なすことができます。

$${K} = 
  \left(
    \begin{array}{ccc}
      k(\vec{x}_{1},\vec{x}_{1}) & k(\vec{x}_{1},\vec{x}_{2}) & \cdots & k(\vec{x}_{1},\vec{x}_{n}) \\
      k(\vec{x}_{2},\vec{x}_{1}) & k(\vec{x}_{2},\vec{x}_{2}) & \cdots & k(\vec{x}_{2},\vec{x}_{n}) \\
      \vdots & \vdots & \ddots & \vdots \\
      k(\vec{x}_{n},\vec{x}_{1}) & k(\vec{x}_{n},\vec{x}_{2}) & \cdots & k(\vec{x}_{n},\vec{x}_{n})
    \end{array}
  \right)
$$

　ここで示されている$k(\vec{x}_{i}, \vec{x}_{j})$は、 **「カーネル関数」** と呼ばれ、数種類存在します。今回は、 **「動径基底関数 (RBF)」** または **「ガウスカーネル」** と呼ばれ以下の式で表されるカーネル関数を使います。この式で2つのデータ $\vec{x}_{i},\vec{x}_{j}$ の類似度を表しています。
 
$$k(\vec{x}_{i},\vec{x}_{j}) \equiv \mathrm{exp}(-\gamma |\vec{x}_{i}-\vec{x}_{j}|^2)$$

　 ${K}$ は ${X}$ と同じように、行が個々のデータを、列が各特徴量（他のデータとの類似度）を示します。  
            
　ここで、$y =  \mathrm{exp}(-\gamma x^2)$ は以下のようなグラフの関数です。

<img src="https://aidemyexcontentspic.blob.core.windows.net/contents-pic/5030_unsupervised_learning/unsupervised_chap3_5.png">

　$\gamma$ を大きくすると、 **より近接するものだけに注目したような特徴量行列** ${K}$ **が作られます** 。

#### 問題

- カーネルトリックについて述べた文として最も適切なものを選んでください。

- データ数n、特徴数dのデータにカーネルトリックを用いると、データ数n、特徴数dのデータができる。
- データ数n、特徴数dのデータにカーネルトリックを用いると、データ数m（≠n）、特徴数k（≠d）のデータができる。
- データ数n、特徴数dのデータにカーネルトリックを用いると、データ数n、特徴数nのデータができる。
- データ数n、特徴数dのデータにカーネルトリックを用いると、データ数d、特徴数dのデータができる。

#### ヒント

- カーネルトリックによって、各データの特徴量数はデータの数と一致するようになります。

#### 解答

- データ数n、特徴数dのデータにカーネルトリックを用いると、データ数n、特徴数nのデータができる。

***

### 3.2.3 カーネルトリックⅡ

　**カーネルトリック**に用いる以下の関数を実装します。  

$$k(\vec{x}_{i},\vec{x}_{j}) \equiv \mathrm{exp}(-\gamma |\vec{x}_{i}-\vec{x}_{j}|^2)$$

　以下のようにして**カーネル行列**を計算できます。
```python
# データ同士の距離の2乗（平方ユークリッド距離）を計算
M = np.sum((X - X[:, np.newaxis])**2, axis=2)
# カーネル行列を計算
K = np.exp(-gamma * M)
```
　ここでデータ間の距離の取得には、numpy配列のブロードキャストという機能（自動的に行列を展開して行列の形を揃え、演算を実行する）を使っています。

#### 問題

- 空欄を埋めて$X$のカーネル行列を作り、カーネル行列の大きさを取得、出力してください。

In [ ]:
import numpy as np

np.random.seed(39)

X = np.random.rand(8, 3)

# ペアごとの平方ユークリッド距離を計算
M =

# カーネル行列を計算
gamma = 15
K =

# ---------------------------
#        ここを書いて下さい
# ---------------------------

print(M)  # 消さないでください。実行結果の確認に使います。
print(K)  # 消さないでください。実行結果の確認に使います。

#### ヒント

K はnumpy配列です。  
numpy配列 A のサイズは以下のようにして取得、表示できます。
```python
print(A.shape)
```

#### 解答例

In [ ]:
import numpy as np

np.random.seed(39)

X = np.random.rand(8, 3)

# ペアごとの平方ユークリッド距離を計算
M = np.sum((X - X[:, np.newaxis])**2, axis=2)

# カーネル行列を計算
gamma = 15
K = np.exp(-gamma * M)

# ---------------------------
print(K.shape)
# ---------------------------

print(M)  # 消さないでください。実行結果の確認に使います。
print(K)  # 消さないでください。実行結果の確認に使います。

***

### 3.2.4 特徴変換

カーネル行列 ${K}$ を元のデータ ${X}$ の代わりとし、 ${K}$ について<b>標準的な主成分分析手法と同様に</b>固有値分解、特徴変換などを行うと、<b>線形分離可能</b>なデータ<b>${X'}$</b>に変換できます。もともと ${K}$ は ${X}$ の特徴量を展開したものですから、 ${K}$ を特徴変換して得られる行列は、 ${X}$ の特徴量を変換した行列として扱うことができます。

これまで学んだことを使って、円状のデータをカーネル主成分分析を用いて特徴変換しましょう。

<img src="https://aidemyexcontentspic.blob.core.windows.net/contents-pic/5030_unsupervised_learning/unsupervised_chap3_4.png">

<b><center>図 3.2.4 特徴変換</center></b>

　発展的な話になりますが、カーネル行列 ${K}$ の固有ベクトルから作る変換行列 ${W}$ は、${X}$ を次元圧縮し要約した ${X'}$ としてそのまま扱うこともできます。ですがここでは先に学んだ主成分分析での手法に沿って特徴変換を行います。

#### 問題

- カーネル主成分分析を用いて、円状のデータを線形分離可能な形に変換してください。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_circles

# データが円状に分布するデータを取得
X, y = make_circles(n_samples=1000, random_state=123, noise=0.1, factor=0.2)

# ペアごとの平方ユークリッド距離を計算してください。
M =

# 対称カーネル行列を計算。γの値は15にしてください。
gamma = 
K =

# カーネル行列から固有対を取得。 numpy.linalg.eighはそれらを固有値の昇順で返す
eigvals, eigvecs = np.linalg.eigh(K)
# 上位k個の固有ベクトル(射影されたサンプル)を収集
W = np.column_stack((eigvecs[:, -1], eigvecs[:, -2]))

# KとWの内積を求めて線形分離可能なデータを獲得してください。
X_kpca =

# 可視化
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 3))
ax1.scatter(X[y == 0, 0], X[y == 0, 1], color="r", marker="^")
ax1.scatter(X[y == 1, 0], X[y == 1, 1], color="b", marker="o")
ax2.scatter(X_kpca[y == 0, 0], X_kpca[y == 0, 1], color="r", marker="^")
ax2.scatter(X_kpca[y == 1, 0], X_kpca[y == 1, 1], color="b", marker="o")
ax1.set_title("circle_data")
ax2.set_title("kernel_pca")
plt.show()

print(M)  # 消さないでください。実行結果の確認に使います。
print(K)  # 消さないでください。実行結果の確認に使います。
print(W)  # 消さないでください。実行結果の確認に使います。
print(X_kpca)  # 消さないでください。実行結果の確認に使います。

#### ヒント

- カーネル行列を作った後、主成分分析の項で学んだのと同じようにカーネル行列の固有値分解を行い、変換行列を求め、カーネル行列に変換行列を掛けあわせることで線形分離可能なデータに変換できます。

#### 解答例

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_circles

# データが円状に分布するデータを取得
X, y = make_circles(n_samples=1000, random_state=123, noise=0.1, factor=0.2)

# ペアごとの平方ユークリッド距離を計算
M = np.sum((X - X[:, np.newaxis])**2, axis=2)

# 対称カーネル行列を計算
gamma = 15
K = np.exp(-gamma * M)

# カーネル行列から固有対を取得。 numpy.linalg.eighはそれらを固有値の昇順で返す
eigvals, eigvecs = np.linalg.eigh(K)
# 上位k個の固有ベクトル(射影されたサンプル)を収集
W = np.column_stack((eigvecs[:, -1], eigvecs[:, -2]))

# KとWの内積を求めて線形分離可能なデータを獲得してください。
X_kpca = K.dot(W)

# 可視化
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 3))
ax1.scatter(X[y == 0, 0], X[y == 0, 1], color="r", marker="^")
ax1.scatter(X[y == 1, 0], X[y == 1, 1], color="b", marker="o")
ax2.scatter(X_kpca[y == 0, 0], X_kpca[y == 0, 1], color="r", marker="^")
ax2.scatter(X_kpca[y == 1, 0], X_kpca[y == 1, 1], color="b", marker="o")
ax1.set_title("circle_data")
ax2.set_title("kernel_pca")
plt.show()

print(M)  # 消さないでください。実行結果の確認に使います。
print(K)  # 消さないでください。実行結果の確認に使います。
print(W)  # 消さないでください。実行結果の確認に使います。
print(X_kpca)  # 消さないでください。実行結果の確認に使います。

***

### 3.2.5 scikit-learnを使ったカーネル主成分分析

　**カーネル主成分分析**は、標準のPCAと同じように `sklearn.decomposition` を使って簡単に実装できます。使い方はほとんど標準のPCAと同様です。引数で圧縮後の次元の数と、標準のPCAにはなかったカーネルの種類を指定することができます。
　
```python
from sklearn.decomposition import KernelPCA
# 今回使うカーネル（動径基底関数）は、 kernel="rbf" で指定できます。
kpca = KernelPCA(n_components=2, kernel="rbf", gamma=15)
X_kpca = kpca.fit_transform(X)
```

　ここでは下図のような月形のデータの分離を行います。次のようにしてデータを取得します。

```python
from sklearn.datasets import make_moons
# 月形データを取得
X, y = make_moons(n_samples=100, random_state=123)
```

<img src="https://aidemyexcontentspic.blob.core.windows.net/contents-pic/5030_unsupervised_learning/unsupervised_chap3_6.png">

#### 問題

- 空欄を埋めてKernelPCAクラスを用いて特徴変換を行い、月形のデータを線形分離可能なデータに変換しましょう。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import make_moons
# KernelPCAをインポート
# ---------------------------

# ---------------------------

# 月形データを取得
X, y = make_moons(n_samples=100, random_state=123)

# KernelPCAクラスをインスタンス化
kpca =
# データXをKernelPCAを用いて変換
X_kpca =

# 可視化
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 3))
ax1.scatter(X[y == 0, 0], X[y == 0, 1], c="r")
ax1.scatter(X[y == 1, 0], X[y == 1, 1], c="b")
ax1.set_title("moon_data")
ax2.scatter(X_kpca[y == 0, 0], X_kpca[y == 0, 1], c="r")
ax2.scatter(X_kpca[y == 1, 0], X_kpca[y == 1, 1], c="b")
ax2.set_title("kernel_PCA")
plt.show()

print(X_kpca)  # 消さないでください。実行結果の確認に使います。

#### ヒント

- KernelPCAクラスを、主成分分析の項で扱ったPCAと同じように適切に使うことで線形分離可能な特徴量に変換できます。 

#### 解答例

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import make_moons
# KernelPCAをインポート
# ---------------------------
from sklearn.decomposition import KernelPCA
# ---------------------------

# 月形データを取得
X, y = make_moons(n_samples=100, random_state=123)

# KernelPCAクラスをインスタンス化
kpca = KernelPCA(n_components=2, kernel="rbf", gamma=15)
# データXをKernelPCAを用いて変換
X_kpca = kpca.fit_transform(X)

# 可視化
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 3))
ax1.scatter(X[y == 0, 0], X[y == 0, 1], c="r")
ax1.scatter(X[y == 1, 0], X[y == 1, 1], c="b")
ax1.set_title("moon_data")
ax2.scatter(X_kpca[y == 0, 0], X_kpca[y == 0, 1], c="r")
ax2.scatter(X_kpca[y == 1, 0], X_kpca[y == 1, 1], c="b")
ax2.set_title("kernel_PCA")
plt.show()

print(X_kpca)  # 消さないでください。実行結果の確認に使います。

***

## 3.3 まとめ問題(提出不要)

　最後に、線形分離ができないデータをカーネルトリックを用いて、SVMにより分類することを考えます。まず、numpyのlogical_xor関数を使ってXORゲート形式のデータを用意します。100個のサンプルにラベル1を割り当て、その他の100個にラベル-1を割り当てます。このデータは線形分離できないため、カーネルトリックを用いてSVMで境界線を引きます。SVMでカーネルトリックを実現するには、scikit-learnのSVCクラスを利用し、パラメータkernel="linear"を"rbf"に置き換えるだけで大丈夫です。
```python
from sklearn.svm import SVC
# カーネルSVMのインスタンスを生成
kernel_svm = SVC(kernel="rbf", C=10.0, random_state=0, gamma=.10)
```

#### 問題

- 以下のコードを完成させてください。

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from matplotlib.colors import ListedColormap
from sklearn.svm import SVC

# 乱数値を設定

# 標準正規分布に従う乱数で200行2列の行列を生成

# 2つの引数に対して排他的論理和を実行
y_xor = np.logical_xor(X_xor[:, 0] > 0, X_xor[:, 1] > 0)
# 排他的論理和の値が真の場合は1、偽の場合は-1を割り当てる
y_xor = np.where(y_xor, 1, -1)
# ラベル1を青のxでプロット

# ラベル-1を赤の四角でプロット


plt.xlim([-3, 3])
plt.ylim([-3, 3])
plt.legend(loc="best")


def plot_decision_regions(X, y, classifier, test_idx=None, resolution=0.02):

    # カラーマップの準備
    markers = ("s", "x", "o", "^", "v")
    colors = ("red", "blue", "lightgreen", "gray", "cyan")
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # 決定領域のプロット
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    # グリッドポイントの生成
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))
    # 各特徴量を1次元配列に変換して予測を実行
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    # 予測結果を元のグリッドポイントのデータサイズに変換
    Z = Z.reshape(xx1.shape)
    # グリッドポイントの等高線のプロット
    plt.contourf(xx1, xx2, Z, alpha=0.4, cmap=cmap)
    # 軸の範囲の設定
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    # クラスごとにサンプルをプロット
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0],
                    y=X[y == cl, 1],
                    alpha=0.6,
                    c=cmap(idx),
                    edgecolor="black",
                    marker=markers[idx],
                    label=cl)

# RBFカーネルによるSVMのインスタンスを生成


svm.fit(X_xor, y_xor)
# 分類領域を可視化
plot_decision_regions(X_xor, y_xor, classifier=svm)
plt.legend(loc="upper left")
plt.show()

#### ヒント

- SVMのインスタンス化の際、引数kernelに"rbf"を指定すると、データに対しカーネルトリックを用いることができます。

#### 解答例

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from matplotlib.colors import ListedColormap
from sklearn.svm import SVC

# 乱数値を設定
np.random.seed(0)
# 標準正規分布に従う乱数で200行2列の行列を生成
X_xor = np.random.randn(200, 2)
# 2つの引数に対して排他的論理和を実行
y_xor = np.logical_xor(X_xor[:, 0] > 0, X_xor[:, 1] > 0)
# 排他的論理和の値が真の場合は1、偽の場合は-1を割り当てる
y_xor = np.where(y_xor, 1, -1)
# ラベル1を青のxでプロット
plt.scatter(X_xor[y_xor == 1, 0], X_xor[y_xor == 1, 1],
            c="b", marker="x", label="1")
# ラベル-1を赤の四角でプロット
plt.scatter(X_xor[y_xor == -1, 0], X_xor[y_xor == -1, 1],
            c="r", marker="s", label="-1")

plt.xlim([-3, 3])
plt.ylim([-3, 3])
plt.legend(loc="best")


def plot_decision_regions(X, y, classifier, test_idx=None, resolution=0.02):

    # カラーマップの準備
    markers = ("s", "x", "o", "^", "v")
    colors = ("red", "blue", "lightgreen", "gray", "cyan")
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # 決定領域のプロット
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    # グリッドポイントの生成
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))
    # 各特徴量を1次元配列に変換して予測を実行
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    # 予測結果を元のグリッドポイントのデータサイズに変換
    Z = Z.reshape(xx1.shape)
    # グリッドポイントの等高線のプロット
    plt.contourf(xx1, xx2, Z, alpha=0.4, cmap=cmap)
    # 軸の範囲の設定
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    # クラスごとにサンプルをプロット
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0],
                    y=X[y == cl, 1],
                    alpha=0.6,
                    c=cmap(idx),
                    edgecolor="black",
                    marker=markers[idx],
                    label=cl)


# RBFカーネルによるSVMのインスタンスを生成
svm = SVC(kernel="rbf", random_state=0, gamma=0.1, C=10.0)
svm.fit(X_xor, y_xor)
# 分類領域を可視化
#plot_decision_regions(X_xor, y_xor, classifier=svm)
#plt.legend(loc="upper left")
plt.show()

#### 解説

　得られたグラフより、SVMでは分離不可能だったデータが、綺麗に分離されていることが確認できます。

***